In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\grand\\Desktop\\End-to-End-Medical-Chatbot-Generative-AI'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
#Extract Data from PDF
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf_file(data='Data/')

In [6]:
#Split data in text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of text chunks:", len(text_chunks))

length of text chunks: 6973


In [8]:
# text_chunks

In [9]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [10]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

C:\Users\grand\AppData\Local\Temp\ipykernel_10516\2610436406.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
c:\Users\grand\Desktop\End-to-End-Medical-Chatbot-Generative-AI\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [14]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"
        
    )
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '299acd8923897e487b819e7cebd561e3', 'date': 'Mon, 21 Jul 2025 02:13:18 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding=embeddings,
)

In [ ]:
#Load Existing index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [ ]:
docsearch

In [ ]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(
    model="llama3.2",
    # model="mistral",  # You can change this to mistral, gemma, etc.
    # model="llama3",
    temperature=0.4,
    max_tokens=500,
    base_url="http://localhost:11434"
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a medical assistant. Answer the user's question using the context provided below. "
    "If the context does not have enough information, say 'I have limited provided information.'\n\n"
    "Make sure to differentiate between similar conditions if asked, and provide clear and factual information.\n\n"
    "CONTEXT:\n{context}\n"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "What is chermotherapy risks?"})
print(response["answer"])

Based on the provided context, I can answer your question. Chermotherapy is not a term mentioned in the context, but I believe you meant to ask about chemotherapy risks.

Chemotherapy is a type of cancer treatment that uses powerful medications to kill cancer cells. While it has been effective in treating many types of cancer, it also carries some risks and side effects.

In terms of birth defects, radiation exposure from the Chernobyl disaster can cause birth defects such as malformed or absent limbs, as mentioned in the context. However, chemotherapy itself is not typically associated with a significant risk of birth defects.

Some common risks and side effects associated with chemotherapy include:

* Nausea and vomiting
* Hair loss (alopecia)
* Fatigue
* Increased risk of infection
* Changes in taste and smell
* Mouth sores and ulcers
* Diarrhea or constipation
* Bone marrow suppression (which can increase the risk of infections)

It's important to note that these risks and side eff